In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Đọc dữ liệu từ file Excel
df1 = pd.read_excel("HomeData.xlsx")

def convert_price(price_str):
    if not isinstance(price_str, str):
        return np.nan
    try:
        if 'tỷ' in price_str:
            return float(price_str.replace(' tỷ', '').replace(',', '.')) * 1e9
        elif 'triệu' in price_str:
            return float(price_str.replace(' triệu', '').replace(',', '.')) * 1e6
        elif 'nghìn' in price_str:
            return float(price_str.replace(' nghìn', '').replace(',', '.')) * 1e3
        else:
            return float(price_str.replace(',', '.'))
    except ValueError:
        print(f"ValueError: Could not convert {price_str} to float.")
        return np.nan

def clean_area(area_str):
    if not isinstance(area_str, str):
        return np.nan
    area_str = area_str.replace(' m²', '')
    if ',' in area_str and '.' in area_str:
        if area_str.count('.') > area_str.count(','):
            area_str = area_str.replace('.', '')
            area_str = area_str.replace(',', '.')
        else:
            area_str = area_str.replace('.', '')
            area_str = area_str.replace(',', '.')
    elif ',' in area_str:
        area_str = area_str.replace(',', '.')
    elif '.' in area_str:
        area_str = area_str.replace('.', '')
    try:
        return float(area_str)
    except ValueError:
        print(f"ValueError: Could not convert {area_str} to float.")
        return np.nan
    
def update_bedroom_toilet(row):
    if row['area'] <= 50:
        if row['bedroom'] == 0:
            row['bedroom'] = 1
        if row['toilet'] == 0:
            row['toilet'] = 1
    elif 50 < row['area'] <= 150:
        if row['bedroom'] == 0:
            row['bedroom'] = 2
        if row['toilet'] == 0:
            row['toilet'] = 2
    elif 150 < row['area'] <= 300:
        if row['bedroom'] == 0:
            row['bedroom'] = 3
        if row['toilet'] == 0:
            row['toilet'] = 3
    elif 300 < row['area'] <= 500:
        if row['bedroom'] == 0:
            row['bedroom'] = 4
        if row['toilet'] == 0:
            row['toilet'] = 4
    elif 500 < row['area'] <= 800:
        if row['bedroom'] == 0:
            row['bedroom'] = 5
        if row['toilet'] == 0:
            row['toilet'] = 5
    elif 800 < row['area'] <= 1000:
        if row['bedroom'] == 0:
            row['bedroom'] = 6
        if row['toilet'] == 0:
            row['toilet'] = 6
    return row

# Xử lý cột 'location' nếu có
if 'location' in df1.columns:
    df1[['district', 'province']] = df1['location'].str.split(', ', expand=True)
    df1.drop(columns=['location'], inplace=True)

# Xóa cột 'price_per_m2'
df1.drop(columns=['price_per_m2'], inplace=True)

# Xóa các hàng có giá là "Giá thỏa thuận"
df1 = df1[df1['price'] != "Giá thỏa thuận"]

# Thay thế giá trị NaN trong cột 'bedroom' và 'toilet' bằng 0
df1['bedroom'] = df1['bedroom'].fillna(0)
df1['toilet'] = df1['toilet'].fillna(0)

# Áp dụng các hàm chuyển đổi
df1['price'] = df1['price'].astype(str).apply(convert_price)
df1['area'] = df1['area'].apply(clean_area)

# Chuyển đổi 'area' sang kiểu số
df1['area'] = pd.to_numeric(df1['area'], errors='coerce')

# Áp dụng hàm cập nhật số phòng ngủ và toilet
df1 = df1.apply(update_bedroom_toilet, axis=1)

# Lọc các hàng có giá trị 'bedroom' và 'toilet' khác 0
df1 = df1[(df1['bedroom'] != 0) | (df1['toilet'] != 0)]

# Xóa các hàng có giá trị thiếu
df1 = df1.dropna()

# Hoặc thay thế giá trị thiếu bằng giá trị trung bình
df1['area'] = df1['area'].fillna(df1['area'].mean())
df1['price'] = df1['price'].fillna(df1['price'].median())

# Tính toán IQR
Q1 = df1[['price', 'area']].quantile(0.25)
Q3 = df1[['price', 'area']].quantile(0.75)
IQR = Q3 - Q1

# Xác định outliers
outliers = (df1[['price', 'area']] < (Q1 - 1.5 * IQR)) | (df1[['price', 'area']] > (Q3 + 1.5 * IQR))
print(outliers.sum())

# Loại bỏ các outliers
df1 = df1[~outliers.any(axis=1)]

# Loại bỏ giá nhà quá cao hoặc quá thấp
df1 = df1[(df1['price'] > 1e5) & (df1['price'] < 1e13)]
df1 = df1[(df1['area'] > 10) & (df1['area'] < 1000)]

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
import joblib

# Mã hóa các đặc trưng phân loại (district, province)
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(df1[['district', 'province']])

# Kết hợp các đặc trưng mã hóa với các đặc trưng số học
X = np.hstack((encoded_features.toarray(), df1[['area', 'bedroom', 'toilet']].values))
y = df1['price']

# Chia tập dữ liệu thành training và testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình hồi quy tuyến tính
model = LinearRegression()
model.fit(X_train, y_train)

# Dự đoán giá nhà mới
new_house = pd.DataFrame({
    'district': ['Quận 2'],
    'province': ['Hồ Chí Minh'],
    'area': [116.87],
    'bedroom': [1],
    'toilet': [2]
})

# Mã hóa dữ liệu nhà mới
encoded_new_house = encoder.transform(new_house[['district', 'province']]).toarray()
new_house_features = np.hstack((encoded_new_house, new_house[['area', 'bedroom', 'toilet']].values))

# Dự đoán giá
predicted_price = model.predict(new_house_features)
print(f"Giá nhà dự đoán: {predicted_price[0]:,.0f} VND")

# Lưu mô hình và encoder
joblib.dump(model, 'model.pkl')
joblib.dump(encoder, 'encoder.pkl')


In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [142]:
# Load the data
df1 = pd.read_csv("muaBanNhaDat.csv")
df1.head()

,price,address,price_m2,rooms,toilets,direction,floors,property_legal_document,house_type,furnishing_sell,width,length,living_size,size
0,"5,50 tỷ",Cập nhật 6 giờ trước,44 triệu/m²,2 phòng,2 phòng,Tây,1.00,Đã có sổ,"Nhà ngõ, hẻm",NaN,NaN,NaN,172.1 m²,NaN
1,"4,50 tỷ","Đường Lê Văn Quới, Phường Bình Hưng Hoà A, Quậ...","112,50 triệu/m²",4 phòng,3 phòng,Đông,4.00,Đã có sổ,"Nhà ngõ, hẻm",Nội thất đầy đủ,NaN,NaN,NaN,NaN
2,"5,50 tỷ","2, Nguyễn Thị Sẳng, Thị trấn Củ Chi, Huyện Củ ...","18,33 triệu/m²",2 phòng,NaN,Đông Nam,1.00,Đã có sổ,"Nhà mặt phố, mặt tiền",Hoàn thiện cơ bản,6 m,NaN,NaN,NaN
3,3 tỷ,"Đường số 22, Phường An Khánh, Quận Ninh Kiều, ...",75 triệu/m²,2 phòng,3 phòng,Đông Nam,2.00,Đã có sổ,"Nhà mặt phố, mặt tiền",Nội thất cao cấp,NaN,NaN,NaN,NaN
4,"3,55 tỷ","Đường Nguyễn Thái Sơn, Phường 4, Quận Gò Vấp, ...","110,94 triệu/m²",2 phòng,2 phòng,Đông Bắc,2.00,Đã có sổ,"Nhà mặt phố, mặt tiền",Hoàn thiện cơ bản,NaN,NaN,NaN,NaN


In [7]:
def convert_price(price_str):
    if not isinstance(price_str, str):
        # Handle cases where the value is not a string (e.g., NaN or numeric values)
        return np.nan
    try:
        if 'tỷ' in price_str:
            return float(price_str.replace(' tỷ', '').replace(',', '.')) * 1e9
        elif 'triệu' in price_str:
            return float(price_str.replace(' triệu', '').replace(',', '.')) * 1e6
        elif 'nghìn' in price_str:
            return float(price_str.replace(' nghìn', '').replace(',', '.')) * 1e3
        else:
            # Handle cases where the value is already numeric
            return float(price_str.replace(',', '.'))
    except ValueError:
        print(f"ValueError: Could not convert {price_str} to float.")
        return np.nan

In [8]:
import re
# Function to clean and convert room and toilet columns
def clean_rooms_toilets(value):
    if pd.isna(value):
        return 0
    value = str(value).lower()
    # Extract number if the string contains 'nhiều hơn'
    if 'nhiều hơn' in value:
        match = re.search(r'\d+', value)
        if match:
            return int(match.group(0))
    # Clean and convert the value
    return int(value.replace('phòng', '').strip())

In [9]:
# Load the data
df1 = pd.read_csv("muaBanNhaDat.csv")
df1.head()

# Handle 'location' column if it exists
if 'address' in df1.columns:
    # Filter out rows where 'address' contains 'Cập nhật'
    df1 = df1[~df1['address'].str.contains('Cập nhật', case=False, na=False)]

    # Split the 'address' into multiple parts based on ', ' and take the last two columns (district and province)
    df1[['district', 'province']] = df1['address'].apply(lambda x: pd.Series(x.split(', ')[-2:]))

    # Remove "Tp " from the beginning of the 'province' column
    df1['province'] = df1['province'].str.replace(r'^Tp\s+', '', regex=True)

    # Drop the 'address' column after splitting
    df1.drop(columns=['address'], inplace=True)

df1 = df1[df1['province'] == "Hồ Chí Minh"]

# Drop rows where price is "Giá thỏa thuận"
df1 = df1[df1['price'] != "Giá thỏa thuận"]

# Apply content to null value
df1['rooms'] = df1['rooms'].apply(clean_rooms_toilets)
df1['toilets'] = df1['toilets'].apply(clean_rooms_toilets)

df1['floors'] = df1['floors'].fillna(0)
df1['house_type'] = df1['house_type'].fillna('')
df1['furnishing_sell'] = df1['furnishing_sell'].fillna('')
df1['province'] = df1['province'].fillna('')

# Apply conversion functions
df1['price'] = df1['price'].astype(str).apply(convert_price)

# Ensure all values in 'price_m2' are strings before extracting the numeric part
df1['price_m2'] = df1['price_m2'].astype(str)

# Extract numeric value from 'price_m2', convert commas to periods, and convert to float
df1['price_m2'] = df1['price_m2'].str.extract(r'([\d,.]+)').replace(',', '.', regex=True).astype(float)

# Convert 'price' and 'price_m2' to numeric, if not already
df1['price'] = pd.to_numeric(df1['price'], errors='coerce')
df1['price_m2'] = pd.to_numeric(df1['price_m2'], errors='coerce')

# Compute the value for 'size' where it is NaN
df1['size'] = df1['size'].fillna(df1['price'] / (df1['price_m2'] * 1000000))

# Làm tròn cột 'size' đến 2 chữ số thập phân
df1['size'] = df1['size'].round(2)

df1.drop(columns=['price_m2'], inplace=True)
df1.drop(columns=['property_legal_document'], inplace=True)
df1.drop(columns=['direction'], inplace=True)
df1.drop(columns=['width'], inplace=True)
df1.drop(columns=['length'], inplace=True)
df1.drop(columns=['living_size'], inplace=True)


# Display the updated DataFrame
df1.head()


,price,rooms,toilets,floors,house_type,furnishing_sell,size,district,province
1,4500000000.00,4,3,4.00,"Nhà ngõ, hẻm",Nội thất đầy đủ,40.00,Quận Bình Tân,Hồ Chí Minh
2,5500000000.00,2,0,1.00,"Nhà mặt phố, mặt tiền",Hoàn thiện cơ bản,300.05,Huyện Củ Chi,Hồ Chí Minh
4,3550000000.00,2,2,2.00,"Nhà mặt phố, mặt tiền",Hoàn thiện cơ bản,32.00,Quận Gò Vấp,Hồ Chí Minh
5,1500000000.00,2,2,2.00,"Nhà ngõ, hẻm",Nội thất đầy đủ,40.00,Huyện Bình Chánh,Hồ Chí Minh
6,68000000000.00,10,6,3.00,"Nhà ngõ, hẻm",Nội thất cao cấp,1074.08,Quận 12,Hồ Chí Minh


In [141]:
# Kiểm tra giá trị thiếu
print(df1.isnull().sum())

price              0
rooms              0
toilets            0
floors             0
house_type         0
furnishing_sell    0
size               0
district           0
dtype: int64


In [10]:
# Convert 'price' and 'size' columns to numeric, forcing errors to NaN
df1['price'] = pd.to_numeric(df1['price'], errors='coerce')
df1['size'] = pd.to_numeric(df1['size'], errors='coerce')

# Drop rows where 'price' or 'size' are NaN (optional, depending on your use case)
df1 = df1.dropna(subset=['price', 'size'])

# Compute IQR
Q1 = df1[['price', 'size']].quantile(0.25)
Q3 = df1[['price', 'size']].quantile(0.75)
IQR = Q3 - Q1

# Identify outliers
outliers = (df1[['price', 'size']] < (Q1 - 1.5 * IQR)) | (df1[['price', 'size']] > (Q3 + 1.5 * IQR))

# Print number of outliers
print(outliers.sum())

# Remove outliers
df1 = df1[~outliers.any(axis=1)]

# Print the shape of the cleaned DataFrame
print(df1.shape)

price    36
size     32
dtype: int64
(474, 9)


In [ ]:
import seaborn as sns

# Vẽ biểu đồ phân phối giá
plt.figure(figsize=(12, 6))
sns.boxplot(x=df1['price'])
plt.title('Boxplot of Price')
plt.show()

# Vẽ biểu đồ phân phối diện tích
plt.figure(figsize=(12, 6))
sns.boxplot(x=df1['area'])
plt.title('Boxplot of Area')
plt.show()


In [119]:
print(df1.describe())
df1.shape

               price  rooms  toilets  floors   size
count         150.00 150.00   150.00  150.00 150.00
mean   4892933333.33   2.99     2.59    2.31  64.20
std    3125006587.20   1.19     1.16    1.35  31.32
min     645000000.00   1.00     0.00    0.00  11.50
25%    2505000000.00   2.00     2.00    1.00  37.25
50%    4300000000.00   3.00     2.00    2.00  64.00
75%    6057500000.00   4.00     3.00    3.00  80.08
max   14000000000.00   8.00     6.00    5.00 171.01


(150, 10)

In [151]:
df1.to_csv('dataset.csv',encoding="utf-8-sig",index=False)

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

df = pd.read_csv('dataset.csv')

# Encode các đặc trưng phân loại (district, province)
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(df[['district', 'province', 'house_type', 'furnishing_sell']])

# Kết hợp các đặc trưng được mã hóa với các đặc trưng số học
X = np.hstack((encoded_features.toarray(), df[['size', 'rooms', 'toilets', 'floors']].values))
y = df['price']

# Chia tập dữ liệu thành training và testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình hồi quy tuyến tính
model = LinearRegression()
model.fit(X_train, y_train)

# Dự đoán giá nhà mới
new_house = pd.DataFrame({
    'district': ['Quận Bình Tân'],
    'province': ['Hồ Chí Minh'],
    'size': [40],
    'rooms': [4],
    'toilets': [3],
    'floors': [4],
    'house_type': ['Nhà ngõ, hẻm'],
    'furnishing_sell': ['Nội thất đầy đủ'],
})

# Encode dữ liệu nhà mới
encoded_new_house = encoder.transform(new_house[['district', 'province', 'house_type', 'furnishing_sell']]).toarray()
new_house_features = np.hstack((encoded_new_house, new_house[['size', 'rooms', 'toilets', 'floors']].values))

# Dự đoán giá
predicted_price = model.predict(new_house_features)
print(f"Giá nhà dự đoán: {predicted_price[0]:,.0f} VND")


Giá nhà dự đoán: 2,992,947,200 VND
